# _Exploratory Data Analysis of the Population and Crop production data._

## Dataset Summary.

There are 2 datasets used in this project; Population dataset and Crops dataset. The population dataset used in this project was created by **THE WORLD BANK** consists of the population of all contries and economies ranging from 1960 to 2019. This dataset was obtained from []() 

# _References_

Data.worldbank.org. 2020. Population, Total | Data. [online] Available at: <https://data.worldbank.org/indicator/SP.POP.TOTL> [Accessed 21 November 2020].